## Heat  equation with Dirichlet conditions
FEniCS tutorial demo program: Heat equation with Dirichlet conditions.
Test problem is chosen to give an exact solution at all nodes of the mesh.

  u'= Laplace(u) + f  in the unit square<br />
  u = u_D             on the boundary<br />
  u = u_0             at t = 0<br />

  u = 1 + x^2 + alpha*y^2 + beta * t <br />
  f = beta - 2 - 2*alpha

In [8]:
import numpy as np
import ufl

from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import fem, mesh, io, plot


### User params && mesh definitions

In [23]:
T = 2.0            # final time
num_steps = 10     # number of time steps
dt = T / num_steps # time step size
alpha = 6          # parameter alpha
beta = 1.2         # parameter beta

# Create mesh and define function space

# Define mesh
nx = ny = 8
domain = mesh.create_rectangle(MPI.COMM_WORLD, [np.array([-2, -2]), np.array([2, 2])], 
                               [nx, ny], mesh.CellType.triangle)
V = fem.FunctionSpace(domain, ("CG", 1))


# mesh = UnitSquareMesh(nx, ny)
# V = ufl.FunctionSpace(mesh, 'P', 1)

# Define boundary condition
u_D = fem.Function(V)
t = 0
u_D.interpolate(lambda x: 1 + x[0]**2 + alpha * x[1]**2 + beta *t )
# u_D = Expression('1 + x[0]*x[0] + alpha*x[1]*x[1] + beta*t',
                 # alpha=alpha, beta=beta, t=0)

    
    
# initial condition
def initial_condition(x):
    return 1 + x[0]*x[0] + 6*x[1]*x[1]
u_n = fem.Function(V)
u_n.name = "u_n"
u_n.interpolate(initial_condition)
    
def boundary(x, on_boundary):
    return on_boundary

def clamped_boundary(x):
    return np.isclose(x[0], 0)

fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(
    domain, fdim, lambda x: np.full(x.shape[1], True, dtype=bool))
bc = fem.dirichletbc(PETSc.ScalarType(0), fem.locate_dofs_topological(V, fdim, boundary_facets), V)
# boundary_facets = mesh.locate_entities_boundary(domain, fdim, clamped_boundary)


# bc = DirichletBC(V, u_D, boundary)
# bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, boundary_facets), V)


### Variational definition of the heat equation

In [50]:

# Define initial value
# u_n = interpolate(u_D, V)
#u_n = project(u_D, V)

# Define variational problem
# u = TrialFunction(V)
# v = TestFunction(V)
f = fem.Constant(domain,beta - 2 - 2*alpha)

u, v = ufl.TrialFunction(V), ufl.TestFunction(V)


# a = u*v*ufl.dx - u_n*v*ufl.dx 
# L = (ufl.inner(f,v) - ufl.dot(ufl.grad(u), ufl.grad(v)) )* dt * ufl.dx
F = u*v*ufl.dx + dt*ufl.dot(ufl.grad(u), ufl.grad(v))*ufl.dx - (u_n + dt*f)*v*ufl.dx

problem = fem.petsc.NonlinearProblem(F, u, [bc])

# problem = fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})

# solve the adjoint vector


ArityMismatch: Adding expressions with non-matching form arguments ('v_0',) vs ('v_0', 'v_1').

### Time stepping routine

In [36]:
# Time-stepping
u = fem.Function(V)
t = 0
# problem = fem.petsc.LinearProblem(F == 0, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
for n in range(num_steps):

    # Update current time
    t += dt
    u_D.t = t

    # Compute solution
    # solve(a == L, u, bc)

    u = problem.solve()

    # Plot solution
    plot(u)

    # Compute error at vertices
    u_e = interpolate(u_D, V)
    error = np.abs(u_e.vector() - u.vector()).max()
    print('t = %.2f: error = %.3g' % (t, error))

    # Update previous solution
    u_n.assign(u)


TypeError: __init__() missing 1 required positional argument: 'L'